In [ ]:
import geopandas
import pyproj
import shapely
import geo_encodings
import numpy as np

In [ ]:
fname = 'data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp'
gdf = geopandas.read_file(fname)

In [ ]:
gdf.head(3)

In [ ]:
mass = None
for rec in gdf.to_dict('records'):
    if rec['NAME'] == 'Massachusetts':
        mass = rec
mass

In [ ]:
def get_projections(lon, lat):

    center_lat = lat
    center_lon = lon
    x0 = 180000
    y0 = 135000

    proj_def = f"""
    +proj=tmerc +lat_0={center_lat} +lon_0={center_lon}
    +k=1.0 +x_0={x0} +y_0={y0} +datum=WGS84 +units=m +no_defs
    """

    ltm_crs = pyproj.CRS.from_proj4(proj_def)
    wgs84_crs = pyproj.CRS.from_epsg(4326)
    proj_forward = pyproj.Transformer.from_crs(wgs84_crs, ltm_crs, always_xy=True).transform
    proj_inverse = pyproj.Transformer.from_crs(ltm_crs, wgs84_crs, always_xy=True).transform

    return proj_forward, proj_inverse

In [ ]:
g0 = mass['geometry']
clon = g0.centroid.xy[0][0]
clat = g0.centroid.xy[1][0]
proj_forward, proj_inverse = get_projections(clon, clat)
g1 = shapely.ops.transform(proj_forward, g0)
g1.bounds

In [ ]:

resolution = 12000
x0, y0, = 0, 0
x1, y1 = 380000, 250000

encoder = geo_encodings.MPPEncoder(
    region=[x0, y0, x1, y1],
    resolution=resolution,
    center=True
)
print('%d elements in encodings' % len(encoder))

In [ ]:
e = encoder.encode(g1).values()
distances = -1.0 * resolution * np.log(e)

In [ ]:
import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter
from geo_encodings import draw_shape


def circle_xy(x0, y0, radius):
    tt = np.linspace(0, 2.0 * 3.14159)
    xx = x0 + radius * np.cos(tt)
    yy = y0 + radius * np.sin(tt)
    return xx, yy


fig = plotly.subplots.make_subplots(1, 2)

# ==============
trace = Scatter(
    x=encoder.ref_x, y=encoder.ref_y, name='reference points', 
    mode='markers', marker={'color': 'gray', 'size': 5, 'symbol': 'cross'}
)
fig.append_trace(trace, 1, 1)
draw_shape(g1, fig, 1, 1, linewidth=2, showlegend=False)
fig['layout']['xaxis1']['range'] = [x0, x1]
fig['layout']['yaxis1']['range'] = [y0, y1]


# ==============
# Draw the circular "exclusion zones" around each reference point.
for i in range(len(encoder)):
    circle_x, circle_y = circle_xy(encoder.ref_x[i], encoder.ref_y[i], distances[i])
    trace = Scatter(
        x=circle_x, y=circle_y, name='polygon', 
        mode='lines', marker={'color': 'magenta'}, 
        fill='toself', fillcolor = 'rgba(255, 0, 255, 0.01)',
        line={'width': 1}, showlegend=False
    )
    fig.append_trace(trace, 1, 2)

# trace = Scatter(
#     x=encoder.ref_x, y=encoder.ref_y, name='reference points', 
#     mode='markers', marker={'color': 'gray', 'size': 5, 'symbol': 'cross'}
# )
# fig.append_trace(trace, 1, 2)

# Add the actual shape.
draw_shape(g1, fig, 1, 2, linewidth=3, color='#444444', showlegend=False)

lo = fig['layout']
lo['width'] = 1100
lo['height'] = 400
lo['xaxis2']['range'] = [x0, x1]
lo['yaxis2']['range'] = [y0, y1]

fig